# Visualisaties dashboard
Deze worden allemaal in Dashboard_tijdreeksen_YvdW_LvdH_2122.py gezet om uiteindelijk in de dashboard te zetten.

VERSCHILLENDE TABBLADEN VOOR:
- DATAVERKENNING TERUGLEVERING
- DATAVERKENNING EXTRA REGRESSORS
- OUTLIERS EN TRAIN-TEST SPLIT
- ELK MODEL ANDER TABBLAD


1. Opdrachtomschrijving e.d.


2. Eerste plot teruglevering incl rolling mean
3. Histogram per seizoen en boxplots per maand
4. Gem per week/maand/jaar met callback
5. ACF en PACF, origineel en gedifferentieerd, adfuller en schatting p en q


6. Visualisaties weer data met rolling mean 
7. Correlatie heatmap
8. Lineplot teruglevering met alle weerdata, callback met dropdown


9. Isolation Forest lineplots
10. ACF en PACF van originele + geinterpoleerd
11. Train-test split lineplot

BIJ ALLE MODELLEN AIC EN MAE
12. Model 1 (FT)
13. Model 2 (zon)
14. Model 3 (zon+transformatie)
15. Model 4 (zon+temp)
16. ? Model 5 (zon+cluster)

In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import dateparser
from datetime import timedelta
import pmdarima as pm
from statsmodels.graphics.gofplots import qqplot
import datetime
from scipy.stats import shapiro

plt.rcParams["figure.figsize"] = (15,8)
plt.rcParams['image.cmap'] = 'Paired'
np.random.seed(42)

In [52]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

In [45]:
ts = pd.read_csv('Data_tijdreeksen.csv', index_col = 'date')
ts.index = pd.to_datetime(ts.index)
ts.head(2)

,gas,stroom,teruglevering,temp,zonneschijn,duur_neerslag,som_neerslag,day_of_week,day_of_week_name,day_of_month,...,season,anomaly_box_zon,anomaly_box_temp,teruglevering_sc,temp_sc,zonneschijn_sc,anom_if_1,anom_if_2,anom_if_3,teruglevering_int_1
date,,,,,,,,,,,,,,,,,,,,,
2021-05-29,1.0,4.0,20.0,116,73,0,0,5,Saturday,29,...,Spring,False,False,1.526239,0.274087,0.649640,1,1,1,20.0
2021-05-28,0.0,3.0,25.0,121,124,0,0,4,Friday,28,...,Spring,False,False,2.189649,0.354985,1.939885,-1,1,1,25.0


In [46]:
ts.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 336 entries, 2021-05-29 to 2020-06-28
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   gas                  336 non-null    float64
 1   stroom               336 non-null    float64
 2   teruglevering        336 non-null    float64
 3   temp                 336 non-null    int64  
 4   zonneschijn          336 non-null    int64  
 5   duur_neerslag        336 non-null    int64  
 6   som_neerslag         336 non-null    int64  
 7   day_of_week          336 non-null    int64  
 8   day_of_week_name     336 non-null    object 
 9   day_of_month         336 non-null    int64  
 10  month                336 non-null    int64  
 11  month_name           336 non-null    object 
 12  season               336 non-null    object 
 13  anomaly_box_zon      336 non-null    bool   
 14  anomaly_box_temp     336 non-null    bool   
 15  teruglevering_sc     

## Dataverkenning teruglevering

#### 2 Teruglevering rolling mean

In [47]:
ts_rolling = ts.rolling(window = 7, center = True).mean()
ts_rolling.dropna().head(3)

,gas,stroom,teruglevering,temp,zonneschijn,duur_neerslag,som_neerslag,day_of_week,day_of_month,month,anomaly_box_zon,anomaly_box_temp,teruglevering_sc,temp_sc,zonneschijn_sc,anom_if_1,anom_if_2,anom_if_3,teruglevering_int_1
date,,,,,,,,,,,,,,,,,,,
2021-05-26,1.142857,6.857143,13.428571,112.142857,52.571429,38.857143,36.142857,3.0,26.0,5.0,0.0,0.0,0.654328,0.211681,0.132819,0.714286,1.0,1.0,13.428571
2021-05-25,1.285714,7.285714,11.714286,110.428571,43.428571,46.857143,43.142857,3.0,25.0,5.0,0.0,0.0,0.426873,0.183945,-0.098485,0.714286,1.0,1.0,11.714286
2021-05-24,1.285714,7.571429,10.142857,111.428571,36.714286,49.285714,46.000000,3.0,24.0,5.0,0.0,0.0,0.218373,0.200124,-0.268349,1.000000,1.0,1.0,10.142857


In [51]:
fig_rolling_tl = go.Figure()

fig_rolling_tl.add_trace(go.Scatter(x=ts.index,
                                    y=ts['teruglevering'],
                                    name='Meting',
                                    line_color='rgb(0,204,204)'
                                   ))

fig_rolling_tl.add_trace(go.Scatter(x=ts_rolling.index,
                                    y=ts_rolling['teruglevering'],
                                    name="Rolling mean (7 dagen)",
                                    line_color='rgb(64,64,64)'
                                   ))

fig_rolling_tl.update_layout(legend=dict(orientation="h",
                                      yanchor="bottom",
                                      y=1.02,
                                      xanchor="right",
                                      x=1),
                             title={
                                 'text': "Electriciteit teruglevering op basis van zonnepanelen in kWh",
                                 'y':0.86,
                                 'x': 0.08},
                             yaxis_title="Waarde (kWh)",
                             xaxis_title="Datum",
#                              margin={"r":0,"t":0,"l":0,"b":0},
                             template="ggplot2"
                            )

fig_rolling_tl.show()

#### 3 Histogrammen per seizoen en boxplots per maand

In [50]:
fig_boxplot = go.Figure()

fig_boxplot.add_trace(go.Box(x=ts["month"], y=ts["teruglevering"],
    #name="Whiskers and Outliers",
    #boxpoints='outliers', # only outliers
    marker_color='rgb(199, 124, 255)',
    line_color='rgb(248,118,109)'
))

# Labels en plaatsing labels
fig_boxplot.update_layout(title={
                             'text': "Boxplots voor elektriciteit teruglevering per maand",
                             'y':0.88,
                             'x': 0.06},
                          margin={"r":0,"l":0,"b":0},
                          yaxis_title="Elektriciteit teruglevering (kWh)",
                          template="ggplot2",
                          width = 800, height = 600,
                          xaxis = dict(tickmode = 'array',
                                       tickvals = [*range(1, 13)],
                                       ticktext = ['Jan', 'Feb', 'Maart', 'April', 'Mei', 'Juni', 'Juli',
                                                   'Aug','Sept', 'Okt', 'Nov', "Dec"]
    )
)

fig_boxplot.show()

In [58]:
ts_spring = ts[ts['season']=="Spring"]
ts_summer = ts[ts['season']=="Summer"]
ts_autumn = ts[ts['season']=="Autumn"]
ts_winter = ts[ts['season']=="Winter"]

In [80]:
fig_hist1 = make_subplots(rows=2, cols=2, start_cell="top-left")

fig_hist1.add_trace(go.Histogram(x=ts_spring['teruglevering'], name = 'Lente'),
                  row=1, col=1)

fig_hist1.add_trace(go.Histogram(x=ts_summer['teruglevering'], name = "Zomer"),
                  row=1, col=2)

fig_hist1.add_trace(go.Histogram(x=ts_autumn['teruglevering'], name = 'Herfst'),
                  row=2, col=1)

fig_hist1.add_trace(go.Histogram(x=ts_winter['teruglevering'], name = 'Winter'),
                  row=2, col=2)

fig_hist1.update_layout(title={
                             'text': "Histogrammen elektriciteit teruglevering per seizoen",
                             'y':0.88,
                             'x': 0.06})

fig_hist1.show()

In [79]:
fig_hist2 = go.Figure()

dict_bins = dict(start=0, end=30, size=3)

fig_hist2.add_trace(go.Histogram(x=ts_spring['teruglevering'], 
                           name = 'Lente', 
                           xbins=dict_bins))

fig_hist2.add_trace(go.Histogram(x=ts_summer['teruglevering'], 
                           name = "Zomer", 
                          xbins=dict_bins))

fig_hist2.add_trace(go.Histogram(x=ts_autumn['teruglevering'], 
                           name = 'Herfst', 
                          xbins=dict_bins))

fig_hist2.add_trace(go.Histogram(x=ts_winter['teruglevering'], 
                           name = 'Winter', 
                          xbins=dict_bins))

# Overlay both histograms
fig_hist2.update_layout(barmode='overlay',
                        title={
                             'text': "Histogrammen elektriciteit teruglevering per seizoen",
                             'y':0.88,
                             'x': 0.06}
                       )
# Reduce opacity to see both histograms
fig_hist2.update_traces(opacity=0.5)
fig_hist2.show()

#### 4 Gemiddelden per dag vd week/dag vd maand/maand v jaar (met callback)

In [92]:
ts_grouped_day = ts.groupby(['day_of_week', 'day_of_week_name'])['teruglevering']\
                                                                                .mean()\
                                                                                    .reset_index()
ts_grouped_month = ts.groupby('day_of_month')['teruglevering'].mean()

ts_grouped_year = ts.groupby(['month', 'month_name'])['teruglevering']\
                                                                    .mean()\
                                                                            .reset_index()

In [98]:
# Callback toevoegen!
fig_mean = px.line(ts_grouped_day, x="day_of_week", y="teruglevering", 
                   title='Gemiddeld elektriciteit teruglevering')
fig_mean.show()

#### 5 ACF's en PACF's (van origineel en gedifferentieerd) en adfuller
Of gewoon seaborn plot erin zetten?

In [111]:
from statsmodels.tsa.stattools import acf, pacf

In [150]:
acf_tl = acf(ts['teruglevering'], nlags = 10)
pacf_tl = pacf(ts['teruglevering'], nlags = 10)

In [144]:
def plot_acf_pacf(type, values_acf, values_pacf):
    fig = make_subplots(rows=2, cols=1)

    for i in range(1, len(values_acf)):
        fig.add_trace(go.Scatter(x=[i,i], 
                                 y=[0,values_acf[i]]
                                ),
                      row = 1, col=1)
        
    for i in range(1, len(values_pacf)):
        fig.add_trace(go.Scatter(x=[i,i], 
                                 y=[0,values_pacf[i]]
                                ),
                      row = 2, col=1)

    fig.update_layout(title={'text': type,
                             'y':0.88,
                             'x': 0.06}
                     )
    return fig

In [145]:
fig = plot_acf_pacf('ACF', acf_tl, pacf_tl)
fig.show()

## Dataverkenning extra variabelen

## Outliers en train-test split